In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

In [2]:
df = pd.read_excel('sales_data.xlsx')

In [5]:
# Remove trailing spaces from column names
df.columns = df.columns.str.strip()

# Extract relevant columns
data = df[['ProductID', 'Quantity', 'Date and Time']]

In [6]:
# Convert 'Date and Time' to datetime and set it as index
data['Date and Time'] = pd.to_datetime(data['Date and Time'])
data.set_index('Date and Time', inplace=True)

C:\Users\Deepak Dhaka\AppData\Local\Temp\ipykernel_9196\4210018612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date and Time'] = pd.to_datetime(data['Date and Time'])


In [7]:
# Resample the data to get total sales per day
sales_per_day = data.groupby(['ProductID']).resample('D').sum()['Quantity']

# Create sequences for each product
sequence_length = 7  # You can adjust this based on your preference
sequences = []

for product_id, sales in sales_per_day.groupby('ProductID'):
    for i in range(len(sales) - sequence_length):
        sequence = sales.iloc[i:i + sequence_length]
        sequences.append((product_id, np.array(sequence)))

In [8]:
# Create features and labels
X = np.array([seq[1] for seq in sequences])
y = np.array([seq[1][-1] for seq in sequences])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the data for CNN
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], sequence_length, 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], sequence_length, 1))

In [13]:
# Build the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(sequence_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))  # Output layer with a single neuron for regression

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train_reshaped, y_train, epochs=20, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Save the trained model
model.save('trained_model.h5')

# Evaluate the model
loss = model.evaluate(X_test_reshaped, y_test)
print(f'Mean Squared Error on Test Data: {loss}')

Epoch 1/20
1/1 [==============================] - 1s 547ms/step - loss: 0.0000e+00 - val_loss: 0.1610
Epoch 2/20
1/1 [==============================] - 0s 29ms/step - loss: 0.0000e+00 - val_loss: 0.1610
Epoch 3/20
1/1 [==============================] - 0s 27ms/step - loss: 0.0000e+00 - val_loss: 0.1610
Epoch 4/20
1/1 [==============================] - 0s 30ms/step - loss: 0.0000e+00 - val_loss: 0.1610
Epoch 5/20
1/1 [==============================] - 0s 32ms/step - loss: 0.0000e+00 - val_loss: 0.1610
Epoch 6/20
1/1 [==============================] - 0s 31ms/step - loss: 0.0000e+00 - val_loss: 0.1610
Epoch 7/20
1/1 [==============================] - 0s 28ms/step - loss: 0.0000e+00 - val_loss: 0.1610
Epoch 8/20
1/1 [==============================] - 0s 28ms/step - loss: 0.0000e+00 - val_loss: 0.1610
Epoch 9/20
1/1 [==============================] - 0s 28ms/step - loss: 0.0000e+00 - val_loss: 0.1610
Epoch 10/20
1/1 [==============================] - 0s 30ms/step - loss: 0.0000e+00 - val_l

In [14]:
# Function to make predictions for a specific date and product
def predict_quantity(product_id, target_date, model, scaler):
    # Extract sales data for the specified product
    product_sales = sales_per_day.loc[product_id]

    # Extract the sequence leading up to the target date
    target_sequence = product_sales.loc[:target_date].tail(sequence_length)

    # If there are not enough historical data, return an error message
    if len(target_sequence) < sequence_length:
        return "Insufficient historical data for prediction"

    # Normalize the data
    scaled_data = scaler.transform(np.array(target_sequence).reshape(1, -1))

    # Reshape the data for CNN
    input_data = scaled_data.reshape((1, sequence_length, 1))

    # Make the prediction
    predicted_quantity = model.predict(input_data)

    # Inverse transform to get the actual predicted quantity
    predicted_quantity = scaler.inverse_transform(predicted_quantity.reshape(-1, 1))

    # Return the result
    return predicted_quantity[0, 0]

# Example: Predict quantity for ProductID 102 on a specific date
product_id_to_predict = 102
target_date_to_predict = pd.Timestamp('2023-10-01')

predicted_quantity = predict_quantity(product_id_to_predict, target_date_to_predict, model, scaler)

print(f'Predicted quantity for ProductID {product_id_to_predict} on {target_date_to_predict}: {predicted_quantity}')

Predicted quantity for ProductID 102 on 2023-10-01 00:00:00: Insufficient historical data for prediction
